In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib
from matplotlib.ticker import FormatStrFormatter
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import json

In [18]:
def crop_center(im:np.array, crop_width: int=50, crop_height: int=120) \
	-> np.array:
	"""
	im: 2D np array, single slice, 3 channels
	"""
	h, w, c = im.shape
	im = im[crop_height : h - crop_height, crop_width : w - crop_width, :]
	return im

In [49]:
def visualize_data(volume, annotations, annotations_auto, slices, output_path, patient_id):
	colors = {0: (1, 0, 0), 1: (0, 0, 1), 2: (0, 1, 0), 3: (1, 0, 0), 4: (0, 1, 1)}
	colors_auto = {0: (1, 0, 0), 1: (0, 0, 0.8), 2: (0.8, 1, 0), 3: (0.5, 0, 0.2), 4: (0, 0.5, 1)}  #for auto
	
	imlist = []
	for i in slices:
		img1 = volume[i]
		img1 = cv2.cvtColor(img1, cv2.COLOR_GRAY2RGB)
		img1_cropped = crop_center(img1)
		img1_cropped = cv2.resize(img1_cropped, (272, 272))
		cv2.imwrite(os.path.join(output_path, f"{patient_id}_{i}_image.jpg"), (img1_cropped * 255).astype(np.uint8))

		mask = np.zeros_like(img1)
		annotations_slice = [item for item in annotations if item[0]==i]
		print(f"annotations for slice {i}: {len(annotations_slice)}")
		if len(annotations_slice)>0:
			for class_idx in [1, 3, 4]:
				contours = [np.array(item[2]).astype(np.int32) for item in annotations_slice if item[1]==class_idx]
				print(f"contours for class {class_idx}: {len(contours)}")
				if len(contours)>0:
					# mask = cv2.drawContours(mask, contours, -1, colors[class_idx], 2, cv2.LINE_8)
					mask = cv2.fillPoly(mask, contours, colors[class_idx], cv2.LINE_8)
		
		annotations_auto_slice = [item for item in annotations_auto if item[0]==i]
		print(f"annotations for slice {i}: {len(annotations_auto_slice)}")
		if len(annotations_auto_slice)>0:
			for class_idx in [2]:
				contours = [np.array(item[2]).astype(np.int32) for item in annotations_auto_slice if item[1]==class_idx]
				print(f"contours for class {class_idx}: {len(contours)}")
				if len(contours)>0:
					# mask = cv2.drawContours(mask, contours, -1, colors[class_idx], 2, cv2.LINE_8)
					mask = cv2.fillPoly(mask, contours, colors_auto[class_idx], cv2.LINE_8)
		
		mask = crop_center(mask)
		mask = cv2.resize(mask, (272, 272))
		cv2.imwrite(os.path.join(output_path, f"{patient_id}_{i}_mask_imputed.jpg"), (mask * 255).astype(np.uint8))

In [51]:
root_path = "../outputs/autocontours_review_processed"
output_path = "../outputs/schematic"
os.makedirs(output_path, exist_ok=True)

dicom_info = [(3, 63)]  # (patient_id, slice_idx)

for patient_id, slice_idx in dicom_info:
	patient_path = os.path.join(root_path, "{0:03d}_CT_EBRT".format(patient_id))
	image = np.load(os.path.join(patient_path, "image.npz"))["volume"]
	seg_manual = json.load(open(os.path.join(patient_path, "manual.json"), "r"))
	seg_auto = json.load(open(os.path.join(patient_path, "auto.json"), "r"))
	visualize_data(image, seg_manual, seg_auto, [slice_idx], output_path, patient_id)

annotations for slice 63: 4
contours for class 1: 0
contours for class 3: 2
contours for class 4: 1
annotations for slice 63: 4
contours for class 2: 1


## random prediction

In [55]:
colors_auto = {0: (0, 0, 0), 1: (0, 0, 0.8), 2: (0.8, 1, 0), 3: (0.5, 0, 0.2), 4: (0, 0.5, 1)}  #for auto

img = (np.random.random((40, 40)) * 4).astype(np.int16)
print(img.max(), img.min())
img_color = np.zeros((40, 40, 3)).astype(np.float32)
for idx, color in colors_auto.items():
    img_color[img==idx] = color

cv2.imwrite(os.path.join(output_path, f"random_pred.jpg"), (img_color * 255).astype(np.uint8))

3 0


True

In [4]:
5*0.2*np.log(0.2)

-1.6094379124341003

In [6]:
np.exp(-1.6)

0.20189651799465538